## Домашняя работа

1. Наборы данных вида Transactions (несколько транзакций на одного клиента) трансформировать в таблицу, где cl_id будут уникальными (соответственно 4000 строк в train и 1000 строк в test
2. Для каждого cl_id будет уникальное целевое событие target_flag, а также уникальный канал привлечения клиента channel_type (клиент привлекается лишь однажды и с самого начала его записи присваивается значение канала привлечения)
3. При агрегации (*pandas.DataFrame.groupby*) по cl_id (или по связке cl_id, channel_type, target_flag) необходимо создавать производные фичи, идеи для таких фичей могут быть следующими:

    - общая сумма транзакций по каждой из trx_category
    - общая сумма транзакции по основным вылютам (напр. выделить рубли, доллары и евро - предположительно, это будут самые крупные категории)
    - общая сумма транзакций по категориям MCC кодов (например, выбрать основные/популярные MCC коды). ВНИМАНИ! Некоторые MCC коды из train могут быть не представлены в test. Про MCC коды в целом: http://www.banki.ru/wikibank/mcc-kod/; Справочник MCC кодов: https://mcc-codes.ru/code; Про некоторые категории кэшбека Росбанка: https://mcc-codes.ru/card/rosbank-sverkh-plus;
    - возможные агрегации по времени суток и дням недели - траты в выходные (праздники) или будни, в ночное время или в рабочее и т.д.
4. **Обязательная часть**: провести первичный анализ данных - посмотреть распределения признаков, выделить самые популярные MCC, помотреть активность клиентов по дням недели/времени, какие категории транзакции (trx_category) наиболее популярны и т.д. Получить инсайты, которые в дальнейшем помогут вам правильно подготовить фичи
5. **Дополнительная часть**: с отобранными фичами и полученными компонентами обучить модель (тип алгоритма на свой вкус, можно начать с линейной) и померить качество на локальном тестовом наборе данных (локальная валидация), который создается в этом ноутбуке. **Метрика оценки качества - ROC AUC**(https://en.wikipedia.org/wiki/Receiver_operating_characteristic)
6. Задания принимаются в виде ноутбука с кодов/картинками выполненной обязательной части + указанием места в leaderboard при решении дополнительной

При возникновении вопросов и для отправки домашнего задания - egsachko@gmail.com или http://fb.com/sachkoe
    

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import numpy as np
from sklearn.metrics import roc_auc_score
#import xgboost
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn import svm
from sklearn.metrics import roc_auc_score, roc_curve

In [2]:
raw_df = pd.read_csv('rosbank_train.csv')
#raw_df.head(10)

In [3]:
#raw_df["MCC"].value_counts(normalize = True)
# 5411    121640 - Бакалейные магазины, супермаркеты
# 6011     54382 - Финансовые учреждения – снятие наличности автоматически
# 5814     41351 - Фастфуд
# 5812     30027 - Места общественного питания, рестораны
# 5499     27237 - Различные продовольственные магазины - нигде более не классифицированные
# 5541     19816 - Станции техобслуживания, с дополнительными услугами или без
# 5912     18728 - Аптеки
# 5999     13073 - Различные магазины и специальные розничные магазины
# 6012     10056 - Финансовые учреждения – торговля и услуги

##### Создаем таблицу df_trans_count со столбцами: суммарные траты, суммарный депозит, разница между этими величинами

In [4]:
sum_income = raw_df[(raw_df['trx_category'].isin(['DEPOSIT','C2C_IN','BACK_TRX']))].groupby(['cl_id'])['amount'].sum()
sum_income = sum_income.rename('sum_income')

sum_outcome = raw_df[ ~ (raw_df['trx_category'].isin(['DEPOSIT','C2C_IN','BACK_TRX']))].groupby(['cl_id'])['amount'].sum()
sum_outcome = sum_outcome.rename('sum_outcome')

in_out_diff  = sum_income - sum_outcome
in_out_diff = in_out_diff.rename('in_out_diff')


target_flag_grouped = raw_df.groupby(['cl_id']).agg('target_flag').first()
target_flag_grouped = target_flag_grouped.rename('target_flag')

channel_type_grouped = raw_df.groupby(['cl_id']).agg('channel_type').first()
channel_type_grouped = channel_type_grouped.rename('channel_type')

df_trans_count = channel_type_grouped.reset_index().set_index('cl_id').join(target_flag_grouped).join(sum_income).join(sum_outcome).join(in_out_diff)
df_trans_count = df_trans_count.reset_index(col_level=1)
df_trans_count.replace({np.nan:0}, inplace=True)
#df_trans_count.head(15)


In [5]:
# Таблица сумма транзакций, минимальное и максимальное значение
#df_cats_agg = raw_df.groupby(['cl_id']).agg({'amount':['sum','min','max']})
#df_cats_agg = df_cats_agg.reset_index(col_level=1)
#df_cats_agg.columns = df_cats_agg.columns.get_level_values(1)
#df_cats_agg.head(5)

#Джойним в таблицу df_trans_count
#df_trans_count_min_max = df_cats_agg.set_index('cl_id').join(sum_income).join(sum_outcome).join(in_out_diff).join(target_flag_grouped)
#df_trans_count_min_max.head(5)

##### Таблицы по тратам по дням и по часам

In [6]:
#Функция для получения транзакций по дням
def transaction_by_day_count(dataset):
    dataset['day_of_transaction'] = dataset['TRDATETIME'].apply( lambda x: (datetime.datetime.strptime(x, '%d%b%y:%H:%M:%S')).strftime('%w') ) 
    tmp_df = dataset.groupby(['cl_id','day_of_transaction'])['day_of_transaction'].size().unstack().fillna(0).reset_index()
    old_columns = [old_col for old_col in tmp_df.columns.tolist() if old_col in dataset['day_of_transaction'].unique()]
    tmp_df.rename(columns={old_col: 'day_' + old_col for old_col in old_columns}, inplace=True)
    return tmp_df

In [7]:
# Функция для получения транзакций по часам
def transaction_by_hour_count(dataset):
    dataset['hour_of_transaction'] = dataset['TRDATETIME'].apply( lambda x: (datetime.datetime.strptime(x, '%d%b%y:%H:%M:%S')).strftime('%H') ) 
    tmp_df = dataset.groupby(['cl_id', 'hour_of_transaction'])['hour_of_transaction'].size().unstack().fillna(0).reset_index()
    old_columns = [old_col for old_col in tmp_df.columns.tolist() if old_col in dataset['hour_of_transaction'].unique()]
    tmp_df.rename(columns={old_col: 'hour_' + old_col for old_col in old_columns}, inplace=True)
    return tmp_df

In [8]:
#Создаём таблицу с тратами по дням
df_trans_day_count = transaction_by_day_count(raw_df)

df_trans_day_count_f = channel_type_grouped.reset_index().set_index('cl_id').join(target_flag_grouped).join(df_trans_day_count.set_index('cl_id'))
df_trans_day_count_f.replace({np.nan:0}, inplace=True)
#df_trans_day_count_f.head(15)

In [9]:
#Создаём таблицу с тратами по часам
df_trans_hour_count = transaction_by_hour_count(raw_df)

df_trans_hour_count_f = channel_type_grouped.reset_index().set_index('cl_id').join(target_flag_grouped).join(df_trans_hour_count.set_index('cl_id'))
df_trans_hour_count_f.replace({np.nan:0}, inplace=True)
#df_trans_hour_count_f.head(15)

##### Создаём таблицу с самыми используемыми MCC - df_trans_mcc

In [10]:
# 5411    121640 - Бакалейные магазины, супермаркеты
# 6011     54382 - Финансовые учреждения – снятие наличности автоматически
# 5814     41351 - Фастфуд
# 5812     30027 - Места общественного питания, рестораны
# 5499     27237 - Различные продовольственные магазины - нигде более не классифицированные
# 5541     19816 - Станции техобслуживания, с дополнительными услугами или без
# 5912     18728 - Аптеки
# 5999     13073 - Различные магазины и специальные розничные магазины
# 6012     10056 - Финансовые учреждения – торговля и услуги

import sqlite3
conn = sqlite3.connect('transactions.db')
raw_df.to_sql('transactions_raw', if_exists='replace', con=conn)
df_trans_mcc = pd.io.sql.read_sql("""
    select
        cl_id,
        sum(case when MCC = 5411 then amount else 0 end) as MCC5411,
        sum(case when MCC = 6011 then amount else 0 end) as MCC6011,
        sum(case when MCC = 5814 then amount else 0 end) as MCC5814,
        sum(case when MCC = 5812 then amount else 0 end) as MCC5812,
        sum(case when MCC = 5499 then amount else 0 end) as MCC5499,
        sum(case when MCC = 5499 then amount else 0 end) as MCC5541,        
        sum(case when MCC = 5912 then amount else 0 end) as MCC5912,
        sum(case when MCC = 5999 then amount else 0 end) as MCC5999,
        sum(case when MCC = 5999 then amount else 0 end) as MCC6012,
        
/*----------------------avgerage amount----------------------------*/  

        avg(case when MCC = 5411 then amount else 0 end) as MCC5411_avg,
        avg(case when MCC = 6011 then amount else 0 end) as MCC6011_avg,
        avg(case when MCC = 5814 then amount else 0 end) as MCC5814_avg,
        avg(case when MCC = 5812 then amount else 0 end) as MCC5812_avg,
        avg(case when MCC = 5499 then amount else 0 end) as MCC5499_avg,
        avg(case when MCC = 5499 then amount else 0 end) as MCC5541_avg,        
        avg(case when MCC = 5912 then amount else 0 end) as MCC5912_avg,
        avg(case when MCC = 5999 then amount else 0 end) as MCC5999_avg,
        avg(case when MCC = 5999 then amount else 0 end) as MCC6012_avg
        
    from transactions_raw
    group by 1
    order by 1
    """, conn)
#df_trans_mcc.head(5)

In [11]:
# Добавляем в таблицу df_trans_mcc столбцы channel_type и target_flag
df_trans_mcc_f = channel_type_grouped.reset_index().set_index('cl_id').join(target_flag_grouped).join(df_trans_mcc.set_index('cl_id'))
df_trans_mcc_f = df_trans_mcc_f.reset_index(col_level=1)
df_trans_mcc_f.replace({np.nan:0}, inplace=True)
#df_trans_mcc_f.head(5)


#### Создаём финальную таблицу

In [ ]:
df_trans_count.head(5)

In [ ]:
df_trans_day_count.head(5)

In [ ]:
#df_trans_hour_count.head(5)

In [ ]:
df_trans_count_f = df_trans_count.set_index('cl_id').join(df_trans_day_count.set_index('cl_id')).join(df_trans_hour_count.set_index('cl_id')).join(df_trans_mcc.set_index('cl_id'))
df_trans_count_f = df_trans_count_f.reset_index(col_level=1)
df_trans_count_f = df_trans_count_f.drop(["channel_type"], axis = 1)
df_trans_count_f.head(5)


In [ ]:
df_trans_count_f.channel_type.unique()

### Backward selection

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df_trans_count_f, test_size=0.2, random_state=42)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [ ]:
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Imputer, LabelBinarizer

num_attribs = train_set.columns.drop(["target_flag"])

num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer', Imputer(strategy="median")),
    ('std_scaler', StandardScaler()),
])

full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
])

In [ ]:
X_train = train_set.drop("target_flag", axis=1)
y_train = train_set["target_flag"].copy()

X_test = test_set.drop("target_flag", axis=1)
y_test = test_set["target_flag"].copy()

In [ ]:
X_train_prepared = full_pipeline.fit_transform(X_train)
X_test_prepared = full_pipeline.transform(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression().fit(X_train_prepared, y_train)
y_pred = lin_reg.predict(X_test_prepared)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
rmse

### Features

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

#### Iteration - 0

In [ ]:
_iteration_cols = train_set.columns.drop('target_flag')

In [ ]:
def get_formula(_cols):
    return 'target_flag ~ ' + " + ".join(_cols)

In [ ]:
lm = smf.ols(get_formula(_iteration_cols), train_set).fit()
rss = np.sum(lm.resid ** 2)
rmse = np.sqrt(rss / len(lm.resid))
print("RMSE:", rmse)
print("R^2:", lm.rsquared)

In [ ]:
get_formula(_iteration_cols)

In [ ]:
lm_res = lm.summary()
lm_res.tables[1]

### Iteration - 1

In [ ]:
_iteration_cols = _iteration_cols.drop('hour_00')
_iteration_cols = _iteration_cols.drop('hour_01')
_iteration_cols = _iteration_cols.drop('hour_02')
_iteration_cols = _iteration_cols.drop('hour_03')
_iteration_cols = _iteration_cols.drop('hour_04')
_iteration_cols = _iteration_cols.drop('hour_05')
_iteration_cols = _iteration_cols.drop('hour_06')
_iteration_cols = _iteration_cols.drop('hour_07')
_iteration_cols = _iteration_cols.drop('hour_08')
_iteration_cols = _iteration_cols.drop('hour_09')
_iteration_cols = _iteration_cols.drop('hour_10')
_iteration_cols = _iteration_cols.drop('hour_11')
_iteration_cols = _iteration_cols.drop('hour_12')
_iteration_cols = _iteration_cols.drop('hour_13')
_iteration_cols = _iteration_cols.drop('hour_14')
_iteration_cols = _iteration_cols.drop('hour_15')
_iteration_cols = _iteration_cols.drop('hour_16')
_iteration_cols = _iteration_cols.drop('hour_17')
_iteration_cols = _iteration_cols.drop('hour_18')
_iteration_cols = _iteration_cols.drop('hour_19')
_iteration_cols = _iteration_cols.drop('hour_20')
_iteration_cols = _iteration_cols.drop('hour_21')
_iteration_cols = _iteration_cols.drop('hour_22')
_iteration_cols = _iteration_cols.drop('hour_23')

In [ ]:
lm = smf.ols(get_formula(_iteration_cols), train_set).fit()
rss = np.sum(lm.resid ** 2)
rmse = np.sqrt(rss / len(lm.resid))
print("RMSE:", rmse)
print("R^2:", lm.rsquared)

In [ ]:
lm_res = lm.summary()
lm_res.tables[1]

### Iteration - 2

In [ ]:
_iteration_cols = _iteration_cols.drop('MCC5814_avg')
_iteration_cols = _iteration_cols.drop('MCC5411_avg')
_iteration_cols = _iteration_cols.drop('MCC5499_avg')
_iteration_cols = _iteration_cols.drop('MCC5541_avg')
_iteration_cols = _iteration_cols.drop('MCC5812_avg')
_iteration_cols = _iteration_cols.drop('MCC5499')
_iteration_cols = _iteration_cols.drop('MCC5541')
_iteration_cols = _iteration_cols.drop('MCC5812')
_iteration_cols = _iteration_cols.drop('MCC5912_avg')
_iteration_cols = _iteration_cols.drop('MCC5411')
_iteration_cols = _iteration_cols.drop('MCC5814')
_iteration_cols = _iteration_cols.drop('MCC6011')

In [ ]:
lm = smf.ols(get_formula(_iteration_cols), train_set).fit()
rss = np.sum(lm.resid ** 2)
rmse = np.sqrt(rss / len(lm.resid))
print("RMSE:", rmse)
print("R^2:", lm.rsquared)

In [ ]:
lm_res = lm.summary()
lm_res.tables[1]

### Iteration - 3

In [ ]:
_iteration_cols = _iteration_cols.drop('day_0')
_iteration_cols = _iteration_cols.drop('day_1')
_iteration_cols = _iteration_cols.drop('day_2')
_iteration_cols = _iteration_cols.drop('day_3')
_iteration_cols = _iteration_cols.drop('day_4')
_iteration_cols = _iteration_cols.drop('day_5')
_iteration_cols = _iteration_cols.drop('day_6')

In [ ]:
lm = smf.ols(get_formula(_iteration_cols), train_set).fit()
rss = np.sum(lm.resid ** 2)
rmse = np.sqrt(rss / len(lm.resid))
print("RMSE:", rmse)
print("R^2:", lm.rsquared)

In [ ]:
lm_res = lm.summary()
lm_res.tables[1]

### Iteration - 4

In [ ]:
_iteration_cols = _iteration_cols.drop('sum_income')

In [ ]:
lm = smf.ols(get_formula(_iteration_cols), train_set).fit()
rss = np.sum(lm.resid ** 2)
rmse = np.sqrt(rss / len(lm.resid))
print("RMSE:", rmse)
print("R^2:", lm.rsquared)

In [ ]:
lm_res = lm.summary()
lm_res.tables[1]

### Roc_auc

In [ ]:
xgb = xgboost.XGBClassifier(max_depth=5, n_jobs=-1)
xgb.fit(X_train, y_train)
y_train_predict = xgb.predict_proba(X_train)[:, 1]
y_test_predict = xgb.predict_proba(X_test)[:, 1]
roc_auc_train = np.round(roc_auc_score(y_train, y_train_predict), 2)
roc_auc_test = np.round(roc_auc_score(y_test, y_test_predict), 2)
print("Train: ", roc_auc_train)
print("Test: ", roc_auc_test)

### Plot

In [ ]:
import matplotlib
fpr_train, tpr_train, thresholds_train = roc_curve(y_train, y_train_predict)
fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_test_predict)

matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)
plt.plot(fpr_train, tpr_train, label='Train ROC AUC {0}'.format(roc_auc_train))
plt.plot(fpr_test, tpr_test, label='Test ROC AUC {0}'.format(roc_auc_test))
plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6))
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('XGB Classifier', size=16)
plt.legend(loc='lower right')
plt.show()

In [ ]:
matplotlib.rcParams['figure.figsize'] = (10, 10)
xgboost.plot_importance(booster=xgb)
plt.savefig('tree_importance_test1.png')